In [ ]:
# ===========================
# Google Colab – Dataset Setup
# ===========================

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Install kagglehub if not installed
!pip install -q kagglehub

# 3. Import
import kagglehub
import shutil
import os

# 4. Download dataset from Kaggle
print("Downloading dataset...")
source_path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Downloaded to:", source_path)

# 5. Target path in your Drive
target_path = "/content/drive/MyDrive/brain_tumor_dataset"

# 6. Remove old folder if exists (optional)
if os.path.exists(target_path):
    print("Removing old dataset folder...")
    shutil.rmtree(target_path)

# 7. Copy dataset to Drive
print("Copying dataset to Google Drive...")
shutil.copytree(source_path, target_path)

print("\n✅ Dataset is ready at:")
print(target_path)


In [ ]:
from google.colab import drive
import os, json, random
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import ImageFolder

from sklearn.metrics import classification_report

drive.mount('/content/drive')

DATA_ROOT = "/content/drive/MyDrive/brain_tumor_dataset"
TRAIN_FOLDER = os.path.join(DATA_ROOT, "Training")
TEST_FOLDER = os.path.join(DATA_ROOT, "Testing")

BATCH_SIZE = 16
NUM_WORKERS = 4
NUM_EPOCHS = 15
LR = 1e-4
WEIGHT_DECAY = 1e-5
RANDOM_SEED = 42
INPUT_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_SAVE_PATH = "/content/drive/MyDrive/brain_tumor_model_best.pth"
LABELS_SAVE = "/content/drive/MyDrive/label_map.json"

train_tf = T.Compose([
    T.RandomResizedCrop(INPUT_SIZE, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_tf = T.Compose([
    T.Resize((INPUT_SIZE, INPUT_SIZE)),
    T.ToTensor(),
    T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

def create_model(n):
    
    m = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
    m.classifier[1] = nn.Linear(m.classifier[1].in_features, n)
    print("Using EfficientNet-B0")
    return m

def run_epoch(model, loader, crit, opt=None):
    train = opt is not None
    model.train() if train else model.eval()
    loss_sum, ok, total = 0, 0, 0
    preds, labels = [], []

    with torch.set_grad_enabled(train):
        for x, y in tqdm(loader, leave=False):
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            loss = crit(out, y)

            if train:
                opt.zero_grad()
                loss.backward()
                opt.step()

            loss_sum += loss.item() * x.size(0)
            p = out.argmax(1)
            ok += (p == y).sum().item()
            total += y.size(0)

            preds += p.cpu().numpy().tolist()
            labels += y.cpu().numpy().tolist()

    return loss_sum/total, ok/total, np.array(preds), np.array(labels)

def main():
    torch.manual_seed(RANDOM_SEED)
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)

    full_ds = ImageFolder(TRAIN_FOLDER, transform=train_tf)
    class_to_idx = full_ds.class_to_idx
    idx_to_class = {str(v): k for k, v in class_to_idx.items()}

    with open(LABELS_SAVE, "w") as f:
        json.dump(idx_to_class, f)

    n = len(full_ds)
    n_val = max(1, int(0.1 * n))
    n_train = n - n_val

    train_ds, val_ds = random_split(
        full_ds, [n_train, n_val],
        generator=torch.Generator().manual_seed(RANDOM_SEED)
    )

    val_ds.dataset.transform = val_tf

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    model = create_model(len(class_to_idx)).to(DEVICE)

    crit = nn.CrossEntropyLoss()
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=2)

    best = 0
    names = [idx_to_class[str(i)] for i in range(len(class_to_idx))]

    for e in range(NUM_EPOCHS):
        print(f"\nEpoch {e+1}/{NUM_EPOCHS}")

        tr_loss, tr_acc, _, _ = run_epoch(model, train_loader, crit, opt)
        va_loss, va_acc, va_p, va_y = run_epoch(model, val_loader, crit)

        print(f"Train loss {tr_loss:.4f} acc {tr_acc:.4f}")
        print(f"Val   loss {va_loss:.4f} acc {va_acc:.4f}")

        sched.step(va_acc)

        print(classification_report(va_y, va_p, target_names=names, zero_division=0))

        if va_acc > best:
            best = va_acc
            torch.save({
                "model_state_dict": model.state_dict(),
                "class_to_idx": class_to_idx,
                "input_size": INPUT_SIZE
            }, MODEL_SAVE_PATH)
            print(f"Saved best model ({best:.4f})")

    if os.path.exists(TEST_FOLDER) and os.listdir(TEST_FOLDER):
        test_ds = ImageFolder(TEST_FOLDER, transform=val_tf)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        t_loss, t_acc, t_p, t_y = run_epoch(model, test_loader, crit)
        print(f"\nTest loss {t_loss:.4f} acc {t_acc:.4f}")
        print(classification_report(t_y, t_p, target_names=names, zero_division=0))
    else:
        print("\nSkipping test set")

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Class map saved to: /content/drive/MyDrive/label_map.json
Classes: {'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Using EfficientNet-B0

=== Epoch 1/15 ===


Train loss: 0.4342  acc: 0.8599
Val   loss: 0.1566  acc: 0.9457

Val classification report:
              precision    recall  f1-score   support

      glioma       0.98      0.88      0.93       128
  meningioma       0.87      0.92      0.89       129
     notumor       0.96      1.00      0.98       149
   pituitary       0.97      0.96      0.97       165

    accuracy                           0.95       571
   macro avg       0.95      0.94      0.94       571
weighted avg       0.95      0.95      0.95       571

Saved best model to /content/drive/MyDrive/brain_tumor_model_best.pth (val_acc=0.9457)

=== Epoch 2/15 ===


train:   0%|          | 0/322 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

train:   0%|          | 0/322 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train loss: 0.0097  acc: 0.9975
Val   loss: 0.0182  acc: 0.9930

Val classification report:
              precision    recall  f1-score   support

      glioma       1.00      0.98      0.99       128
  meningioma       0.98      1.00      0.99       129
     notumor       0.99      1.00      0.99       149
   pituitary       1.00      0.99      0.99       165

    accuracy                           0.99       571
   macro avg       0.99      0.99      0.99       571
weighted avg       0.99      0.99      0.99       571


=== Epoch 15/15 ===


train:   0%|          | 0/322 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train loss: 0.0101  acc: 0.9969
Val   loss: 0.0164  acc: 0.9947

Val classification report:
              precision    recall  f1-score   support

      glioma       1.00      0.99      1.00       128
  meningioma       0.99      1.00      1.00       129
     notumor       0.99      1.00      0.99       149
   pituitary       1.00      0.99      0.99       165

    accuracy                           0.99       571
   macro avg       0.99      1.00      0.99       571
weighted avg       0.99      0.99      0.99       571


Running final evaluation on test set...


Test loss: 0.0217  acc: 0.9954
Test classification report:
              precision    recall  f1-score   support

      glioma       1.00      0.99      0.99       300
  meningioma       0.98      1.00      0.99       306
     notumor       1.00      1.00      1.00       405
   pituitary       1.00      0.99      0.99       300

    accuracy                           1.00      1311
   macro avg       1.00      1.00      1.00      1311
weighted avg       1.00      1.00      1.00      1311

